In [2]:
import pandas as pd
import numpy as np
import string as st
from afinn import Afinn
# import matplotlib.pylab as plt
# import seaborn as sns
# plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',800)
df_rev=pd.read_csv('../Data/clean_googleplaystore_user_reviews.csv')
df_app=pd.read_csv('../Data/cleaned_googleplaystore.csv')


In [ ]:
df_app

In [ ]:
df_app[['Content Rating','Type','Price']].query("Type=='Paid'").groupby('Content Rating').mean('Price')

In [ ]:
df_rev.info()

TASK_2 - AFINN METHOD (VAIBHAV)

In [3]:
af = Afinn()

# Compute sentiment scores and categories
sentiment_scores = [af.score(str(text)) for text in df_rev.Translated_Review]
sentiment_category = ['positive' if score > 0 else 'negative' if score < 0 else 'neutral' for score in sentiment_scores]
df = pd.DataFrame([sentiment_scores, sentiment_category]).T # .T: swap rows and cols
df.columns = ['AFINN_sentiment_score', 'AFINN_sentiment_category']
df_rev  = pd.concat([df_rev, df], axis=1)
df_rev

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,13.0,positive
1,10 Best Foods for You,This help eating healthy exercise regular basis,4.0,positive
2,10 Best Foods for You,NOREVIEW,0.0,neutral
3,10 Best Foods for You,Works great especially going grocery store,3.0,positive
4,10 Best Foods for You,Best idea us,3.0,positive
...,...,...,...,...
41705,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41706,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41707,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41708,Housing-Real Estate & Property,NOREVIEW,0.0,neutral


TASK_2 - NP METHOD (ONUR)

In [6]:
df_p=pd.read_excel('../Data/p.xlsx',header=None)
df_n=pd.read_excel('../Data/n.xlsx',header=None)

np_dict=dict.fromkeys(list(df_p[0]), 1)
np_dict.update(dict.fromkeys(list(df_n[0]), -1))
len(np_dict)

6785

In [4]:
df_rev['NP_sentiment_score']=pd.NA
df_rev['NP_sentiment_category']=pd.NA

In [6]:
df_rev.head()

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category,NP_sentiment_score,NP_sentiment_category
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,13.0,positive,<NA>,<NA>
1,10 Best Foods for You,This help eating healthy exercise regular basis,4.0,positive,<NA>,<NA>
2,10 Best Foods for You,NOREVIEW,0.0,neutral,<NA>,<NA>
3,10 Best Foods for You,Works great especially going grocery store,3.0,positive,<NA>,<NA>
4,10 Best Foods for You,Best idea us,3.0,positive,<NA>,<NA>


In [7]:
# import logging

# logging.basicConfig(filename='log.txt',
#                     filemode='a',
#                     format='%(message)s',
#                     level=logging.ERROR)
def sentiment_category(s_score):
    if s_score<0:
        return 'Negative'
    elif s_score>0:
        return 'Positive'
    else:
        return 'Neutral'
for i in range(len(df_rev['Translated_Review'])):
    sentiment_score=0
    review=df_rev['Translated_Review'][i].translate(str.maketrans('','',st.punctuation)).lower().split()
    for word in review: #range(len(review)):
        try:
            sentiment_score+=np_dict[word]
        except(KeyError):
            # logging.error(f'{review[z]}')
            pass
    df_rev['NP_sentiment_score'][i]=sentiment_score
    df_rev['NP_sentiment_category'][i]=sentiment_category(sentiment_score)

df_rev

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category,NP_sentiment_score,NP_sentiment_category
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,13.0,positive,4,Positive
1,10 Best Foods for You,This help eating healthy exercise regular basis,4.0,positive,1,Positive
2,10 Best Foods for You,NOREVIEW,0.0,neutral,0,Neutral
3,10 Best Foods for You,Works great especially going grocery store,3.0,positive,2,Positive
4,10 Best Foods for You,Best idea us,3.0,positive,1,Positive
...,...,...,...,...,...,...
41705,Housing-Real Estate & Property,NOREVIEW,0.0,neutral,0,Neutral
41706,Housing-Real Estate & Property,NOREVIEW,0.0,neutral,0,Neutral
41707,Housing-Real Estate & Property,NOREVIEW,0.0,neutral,0,Neutral
41708,Housing-Real Estate & Property,NOREVIEW,0.0,neutral,0,Neutral


In [ ]:
df_rev['NP_sentiment_category'].value_counts()


In [ ]:
df_rev['AFINN_sentiment_category'].value_counts().sort_values()


TASK_3 -for paid apps only list the top 5 highest and lowest sentiment numbers for afinn column with the name of the app and the app category

In [8]:
df_rev=df_rev.merge(df_app[['App','Category','Type']],on='App')

In [9]:
df_rev[df_rev['Type']=='Paid'][['Category','App','AFINN_sentiment_score']].groupby(['Category','App']).agg('mean')


AFINN_sentiment_score
Category        App                                                  
FAMILY          Dr. Panda & Toto's Treehouse                 5.272727
GAME            Call of Duty:Black Ops Zombies              -0.025000
                Farming Simulator 18                         4.000000
MEDICAL         Diabetes & Diet Tracker                      4.232558
PERSONALIZATION Beautiful Widgets Pro                        2.275000
                HD Widgets                                   2.125000

In [ ]:
df_rev.groupby(['Type','App'])['Type'].value_counts()


In [ ]:
list(df_rev['App'].unique()), sorted(list(df_app.query("Type=='Paid'")['App'].unique()))

In [ ]:
df_rev.head(100)

In [ ]:
df_app[df_app['Type']=='Paid']['App'].unique()

TASK_4 - what is the best category according to afinn sentiment values

In [13]:
df_rev[['Category','AFINN_sentiment_score']].groupby('Category').agg('mean').sort_values(by='AFINN_sentiment_score',ascending=False)

,AFINN_sentiment_score
Category,
PARENTING,2.881250
ART_AND_DESIGN,2.576190
MEDICAL,2.372627
HEALTH_AND_FITNESS,2.297056
BOOKS_AND_REFERENCE,2.202778
FOOD_AND_DRINK,2.114187
PERSONALIZATION,1.976870
COMICS,1.975000
AUTO_AND_VEHICLES,1.948310
